In [1]:
import polars as pl
import sqlite3

### Helper function to make binary columns human readable

In [3]:
# Make binary columns human readable
def encode_binary_columns(df, encoding='hex'):
    binary_columns = [col for col in df.columns if df[col].dtype == pl.Binary]
    for col in binary_columns:
        df = df.with_columns(
            pl.col(col).bin.encode(encoding)
        )
    return df

### Make dataframes from the tables in the sqlite database file (only need to do this once)

In [4]:
# Make a connection
conn = sqlite3.connect('fedimint-observer.db')

# Get a list of table names in the database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' OR type='view';")
table_names = [row[0] for row in cursor.fetchall()]

# Write a dataframe for each table
for table_name in table_names:
    df = pl.read_database(query=f"SELECT * FROM {table_name}", connection=conn)
    encode_binary_columns(df).write_parquet(f'./dfs/{table_name}.parquet')    